# Import & Load disabilities data

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/health/disability/'
local_path = '../data/'

### Imports

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load death causes

In [4]:
df = download_file(remote_path,local_path,'disability_data_adults.csv')
df['year'] = 2015
df = df.astype({'region':'int','sex':'int','disability_grade':'int','weight':'int','year':'int'})
list(df)

Downloading... http://pacha.datawheel.us/health/disability/disability_data_adults.csv
Encoding: ascii


['region', 'sex', 'disability_grade', 'weight', 'year']

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'health', 'fact_disability')

DROP TABLE IF EXISTS health.fact_disability;
CREATE TABLE "health"."fact_disability" (
"region" INTEGER,
  "sex" INTEGER,
  "disability_grade" INTEGER,
  "weight" INTEGER,
  "year" INTEGER
)
COPY "health"."fact_disability" ("region","sex","disability_grade","weight","year") FROM STDIN WITH CSV HEADER DELIMITER ',';


In [6]:
engine.execute("""
CREATE INDEX fact_disability_region_id 
ON health.fact_disability (region)
""")

engine.execute("""
ALTER TABLE health.fact_disability
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE health.fact_disability
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = health.fact_disability.year
      AND dim_date.month_of_year = 1
      AND dim_date.day_of_month = 1
""")

### Related dim

In [7]:
d1 = download_file(remote_path,local_path,'disability_grade_id.csv')
print (inline_table_xml(d1, 'disability_grade', 'disability_grade_id', 'disability_grade'))

Downloading... http://pacha.datawheel.us/health/disability/disability_grade_id.csv
Encoding: ascii

<InlineTable alias="disability_grade">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">sin discapacidad</Value>
      <Value column="description">1</Value>
      <Value column="es_description">1</Value>
    </Row>
  <Row>
      <Value column="id">leve a moderada</Value>
      <Value column="description">2</Value>
      <Value column="es_description">2</Value>
    </Row>
  <Row>
      <Value column="id">severa</Value>
      <Value column="description">3</Value>
      <Value column="es_description">3</Value>
    </Row>
  </Rows>
</InlineTable>
    
